In [1]:
import librosa
import soundfile as sf
from datasets import Dataset, Audio, DatasetDict
from tqdm import tqdm
import pandas as pd
import yaml
import os

In [2]:
!pwd

/home/cobrayyxx/coding-coding/kreasof/IWSLT-2025/low-resource/bem-eng/bigc/data/bem


In [195]:
df_train = pd.read_csv("splits/train.tsv",sep='\t')

In [196]:
df_train.head()

,pair_id,image,sentence_id,image_id,audio_id,sentence,translation,speaker_id,gender,duration,sample_rate,audio_format,recording_lang,native_lang
0,26.0,2688190,1,1000092795.jpg,26_8819_0_261_01_211112-010515_bem_8f1_elicit_...,Abalumendo babili nabeminina bale ikata amabul...,these men are standing caching tree leaves,261,male,5.739,16000,vnd.wave,bem,bem
1,38.0,38118474,1,1000523639.jpg,38_11847_4_381_01_211114-161237_bem_ba5_elicit...,Cifwile uyu ou bali nankwe eulemulanga ifyakuc...,It must be that the person they are with is sh...,381,male,4.265,16000,vnd.wave,bem,bem
2,24.0,2426450,1,1000919630.jpg,24_2645_0_241_01_211110-143805_bem_150_elicit_...,Umulumendo ekele pacipuna elo kumaboko ekete i...,A young man sitted on the chair has a red doll...,241,female,6.624,16000,vnd.wave,bem,bem
3,8.0,855581,2,1001773457.jpg,8_5558_1_82_02_211105-023723_bem_901_elicit_0.wav,Shili mu musebo,They are on the road.,82,female,3.993,16000,vnd.wave,bem,bem
4,35.0,35111590,1,1001896054.jpg,35_11159_0_351_01_220118-193228_bem_ba5_elicit...,Umwaume uwabuuta inkanda aletekenya icimbaya m...,One white man is driving a tractor,351,male,5.581,16000,vnd.wave,bem,bem


In [197]:
df_val = pd.read_csv("splits/valid.tsv",sep='\t')

In [198]:
df_val.head()

,pair_id,image,sentence_id,image_id,audio_id,sentence,translation,speaker_id,gender,duration,sample_rate,audio_format,recording_lang,native_lang
0,36.0,3654663,2,1018148011.jpg,36_5466_3_362_02_211113-115353_bem_dbf_elicit_...,cacine wasosa mune nalimo kutali yaleya mune,"yes it's true, it seems it's going very far",362,female,6.329,16000,vnd.wave,bem,bem
1,17.0,1777934,1,103205630.jpg,17_7793_4_171_01_211109-061925_bem_947_elicit_...,Ku cende bali takwaba ififntu fimbi kwabafye i...,There where at a place where there was nothing...,171,male,7.010,16000,vnd.wave,bem,toi
2,14.0,1472070,1,1056873310.jpg,14_7207_0_141_01_211112-120659_bem_f08_elicit_...,Imbwa shibili iya buta neyafita shilelwa nokwa...,"Two dogs, a white one and a black one are play...",141,female,9.052,16000,vnd.wave,bem,bem
3,32.0,32102771,2,1092437557.jpg,32_10277_1_322_02_220121-044029_bem_b40_elicit...,Bushe uyu umukashana uwakwata inkanda iya buut...,How is this white girl looking?,322,female,7.600,16000,vnd.wave,bem,bem
4,12.0,1229440,1,109671650.jpg,12_2944_0_121_01_211031-190209_bem_bd6_elicit_...,umulumendo naiminina pelibwe,a young man is standing on the rock,121,male,6.148,16000,vnd.wave,bem,lam


In [199]:
df_test = pd.read_csv("splits/test.tsv",sep='\t')

In [200]:
df_test.head()

,pair_id,image,sentence_id,image_id,audio_id,sentence,translation,speaker_id,gender,duration,sample_rate,audio_format,recording_lang,native_lang
0,15.0,1574743,2,1007129816.jpg,15_7474_3_152_02_211030-180248_bem_8cc_elicit_...,nafwala na amakalashi ku menso,he is wearing glasses as well,152,female,4.061,16000,vnd.wave,bem,bem
1,10.0,1023450,1,1009434119.jpg,10_2345_0_101_01_211028-193045_bem_ddd_elicit_...,"Imbwa iyafonka pamoona, ilebutuka palunkoto lw...",A dog with a wide nose is running on the lawns...,101,male,7.986,16000,vnd.wave,bem,bem
2,34.0,3450263,2,101362133.jpg,34_5026_3_342_02_211113-233703_bem_50c_elicit_...,Akamwanakashi nakemya ukuulu mumuulu ukulwisha...,She has her leg in the air attempting to break...,342,male,5.354,16000,vnd.wave,bem,lam
3,NaN,1492,1,102617084.jpg,149_2_01_210928-115147_bem_dcc_elicit_1.wav,Kunuma yabo kuli notu ma motoka tulya ba bonfy...,There are also small vehicles that they use in...,11,male,9.528,16000,vnd.wave,bem,bem
4,36.0,36114041,2,10287332.jpg,36_11404_1_362_02_220227-135700_bem_d0b_elicit...,abaume Bali pa mutenge yanganda umo afwele ish...,"Of the men on the roof of the house, ine is we...",362,male,11.910,16000,vnd.wave,bem,bem


In [201]:
df_train = df_train[["audio_id","sentence","translation","speaker_id"]]

In [202]:
df_train.head()

,audio_id,sentence,translation,speaker_id
0,26_8819_0_261_01_211112-010515_bem_8f1_elicit_...,Abalumendo babili nabeminina bale ikata amabul...,these men are standing caching tree leaves,261
1,38_11847_4_381_01_211114-161237_bem_ba5_elicit...,Cifwile uyu ou bali nankwe eulemulanga ifyakuc...,It must be that the person they are with is sh...,381
2,24_2645_0_241_01_211110-143805_bem_150_elicit_...,Umulumendo ekele pacipuna elo kumaboko ekete i...,A young man sitted on the chair has a red doll...,241
3,8_5558_1_82_02_211105-023723_bem_901_elicit_0.wav,Shili mu musebo,They are on the road.,82
4,35_11159_0_351_01_220118-193228_bem_ba5_elicit...,Umwaume uwabuuta inkanda aletekenya icimbaya m...,One white man is driving a tractor,351


In [203]:
df_val = df_val[["audio_id","sentence","translation","speaker_id"]]
df_val.head()

,audio_id,sentence,translation,speaker_id
0,36_5466_3_362_02_211113-115353_bem_dbf_elicit_...,cacine wasosa mune nalimo kutali yaleya mune,"yes it's true, it seems it's going very far",362
1,17_7793_4_171_01_211109-061925_bem_947_elicit_...,Ku cende bali takwaba ififntu fimbi kwabafye i...,There where at a place where there was nothing...,171
2,14_7207_0_141_01_211112-120659_bem_f08_elicit_...,Imbwa shibili iya buta neyafita shilelwa nokwa...,"Two dogs, a white one and a black one are play...",141
3,32_10277_1_322_02_220121-044029_bem_b40_elicit...,Bushe uyu umukashana uwakwata inkanda iya buut...,How is this white girl looking?,322
4,12_2944_0_121_01_211031-190209_bem_bd6_elicit_...,umulumendo naiminina pelibwe,a young man is standing on the rock,121


In [204]:
df_test = df_test[["audio_id","sentence","translation","speaker_id"]]
df_test.head()

,audio_id,sentence,translation,speaker_id
0,15_7474_3_152_02_211030-180248_bem_8cc_elicit_...,nafwala na amakalashi ku menso,he is wearing glasses as well,152
1,10_2345_0_101_01_211028-193045_bem_ddd_elicit_...,"Imbwa iyafonka pamoona, ilebutuka palunkoto lw...",A dog with a wide nose is running on the lawns...,101
2,34_5026_3_342_02_211113-233703_bem_50c_elicit_...,Akamwanakashi nakemya ukuulu mumuulu ukulwisha...,She has her leg in the air attempting to break...,342
3,149_2_01_210928-115147_bem_dcc_elicit_1.wav,Kunuma yabo kuli notu ma motoka tulya ba bonfy...,There are also small vehicles that they use in...,11
4,36_11404_1_362_02_220227-135700_bem_d0b_elicit...,abaume Bali pa mutenge yanganda umo afwele ish...,"Of the men on the roof of the house, ine is we...",362


In [205]:
df_train.shape

(82371, 4)

In [206]:
df_val.shape

(2782, 4)

In [207]:
df_test.shape

(2779, 4)

# Add path to audio_id

In [208]:
def full_audio_path(row):
    full_audio_path = os.path.join("audio", row["audio_id"])
    return full_audio_path

## train

In [209]:
df_train["audio"] = df_train.apply(full_audio_path, axis=1)

In [210]:
audio_col = df_train.pop(df_train.columns[-1])
df_train.insert(0, audio_col.name, audio_col)

In [211]:
df_train.head()

,audio,audio_id,sentence,translation,speaker_id
0,audio/26_8819_0_261_01_211112-010515_bem_8f1_e...,26_8819_0_261_01_211112-010515_bem_8f1_elicit_...,Abalumendo babili nabeminina bale ikata amabul...,these men are standing caching tree leaves,261
1,audio/38_11847_4_381_01_211114-161237_bem_ba5_...,38_11847_4_381_01_211114-161237_bem_ba5_elicit...,Cifwile uyu ou bali nankwe eulemulanga ifyakuc...,It must be that the person they are with is sh...,381
2,audio/24_2645_0_241_01_211110-143805_bem_150_e...,24_2645_0_241_01_211110-143805_bem_150_elicit_...,Umulumendo ekele pacipuna elo kumaboko ekete i...,A young man sitted on the chair has a red doll...,241
3,audio/8_5558_1_82_02_211105-023723_bem_901_eli...,8_5558_1_82_02_211105-023723_bem_901_elicit_0.wav,Shili mu musebo,They are on the road.,82
4,audio/35_11159_0_351_01_220118-193228_bem_ba5_...,35_11159_0_351_01_220118-193228_bem_ba5_elicit...,Umwaume uwabuuta inkanda aletekenya icimbaya m...,One white man is driving a tractor,351


In [212]:
for idx, row in df_train.head().iterrows():
    if row["audio"].split("/")[-1] != row["audio_id"]:
        print(row["audio_id"])


In [213]:
df_train.pop("audio_id")

0        26_8819_0_261_01_211112-010515_bem_8f1_elicit_...
1        38_11847_4_381_01_211114-161237_bem_ba5_elicit...
2        24_2645_0_241_01_211110-143805_bem_150_elicit_...
3        8_5558_1_82_02_211105-023723_bem_901_elicit_0.wav
4        35_11159_0_351_01_220118-193228_bem_ba5_elicit...
                               ...                        
82366    12_2799_2_121_01_211026-021325_bem_bd6_elicit_...
82367    26_8915_3_262_02_211112-133156_bem_dcc_elicit_...
82368    26_8915_0_261_01_211112-051313_bem_8f1_elicit_...
82369    26_8915_4_261_01_211112-051313_bem_8f1_elicit_...
82370    26_8915_2_261_01_211112-051313_bem_8f1_elicit_...
Name: audio_id, Length: 82371, dtype: object

In [214]:
df_train.head()

,audio,sentence,translation,speaker_id
0,audio/26_8819_0_261_01_211112-010515_bem_8f1_e...,Abalumendo babili nabeminina bale ikata amabul...,these men are standing caching tree leaves,261
1,audio/38_11847_4_381_01_211114-161237_bem_ba5_...,Cifwile uyu ou bali nankwe eulemulanga ifyakuc...,It must be that the person they are with is sh...,381
2,audio/24_2645_0_241_01_211110-143805_bem_150_e...,Umulumendo ekele pacipuna elo kumaboko ekete i...,A young man sitted on the chair has a red doll...,241
3,audio/8_5558_1_82_02_211105-023723_bem_901_eli...,Shili mu musebo,They are on the road.,82
4,audio/35_11159_0_351_01_220118-193228_bem_ba5_...,Umwaume uwabuuta inkanda aletekenya icimbaya m...,One white man is driving a tractor,351


## val

In [215]:
df_val["audio"] = df_val.apply(full_audio_path, axis=1)

In [216]:
audio_col = df_val.pop(df_val.columns[-1])
df_val.insert(0, audio_col.name, audio_col)

In [217]:
for idx, row in df_val.head().iterrows():
    if row["audio"].split("/")[-1] != row["audio_id"]:
        print(row["audio_id"])

In [218]:
df_val.pop("audio_id")

0       36_5466_3_362_02_211113-115353_bem_dbf_elicit_...
1       17_7793_4_171_01_211109-061925_bem_947_elicit_...
2       14_7207_0_141_01_211112-120659_bem_f08_elicit_...
3       32_10277_1_322_02_220121-044029_bem_b40_elicit...
4       12_2944_0_121_01_211031-190209_bem_bd6_elicit_...
                              ...                        
2777    19_1282_2_191_01_211108-211219_bem_665_elicit_...
2778        12537_4_01_211102-173234_bem_142_elicit_2.wav
2779        12537_3_02_211102-173208_bem_dcc_elicit_1.wav
2780        12537_2_01_211102-173234_bem_142_elicit_1.wav
2781        12537_1_02_211102-173208_bem_dcc_elicit_0.wav
Name: audio_id, Length: 2782, dtype: object

In [219]:
df_val

,audio,sentence,translation,speaker_id
0,audio/36_5466_3_362_02_211113-115353_bem_dbf_e...,cacine wasosa mune nalimo kutali yaleya mune,"yes it's true, it seems it's going very far",362
1,audio/17_7793_4_171_01_211109-061925_bem_947_e...,Ku cende bali takwaba ififntu fimbi kwabafye i...,There where at a place where there was nothing...,171
2,audio/14_7207_0_141_01_211112-120659_bem_f08_e...,Imbwa shibili iya buta neyafita shilelwa nokwa...,"Two dogs, a white one and a black one are play...",141
3,audio/32_10277_1_322_02_220121-044029_bem_b40_...,Bushe uyu umukashana uwakwata inkanda iya buut...,How is this white girl looking?,322
4,audio/12_2944_0_121_01_211031-190209_bem_bd6_e...,umulumendo naiminina pelibwe,a young man is standing on the rock,121
...,...,...,...,...
2777,audio/19_1282_2_191_01_211108-211219_bem_665_e...,"Aba banensu basompa imyona nabo, uyu muntu imi...","He has a pointed nose, he looks like a lady",191
2778,audio/12537_4_01_211102-173234_bem_142_elicit_...,Nibamona mune ala uyu umupungu walilelema kabi...,"I have seen it, the rice looks and green, this...",11
2779,audio/12537_3_02_211102-173208_bem_dcc_elicit_...,Mukumona kwandi mupunga uyu balebyala kumona f...,"This should be rice, it's clear because it's h...",12
2780,audio/12537_2_01_211102-173234_bem_142_elicit_...,Cishinka sana balebomfye n'gombe pakulima kabi...,Yes they are using cows for ploughing and this...,11


## test

In [220]:
df_test["audio"] = df_test.apply(full_audio_path, axis=1)

In [221]:
audio_col = df_test.pop(df_test.columns[-1])
df_test.insert(0, audio_col.name, audio_col)

In [222]:
for idx, row in df_test.head().iterrows():
    if row["audio"].split("/")[-1] != row["audio_id"]:
        print(row["audio_id"])

In [223]:
df_test.pop("audio_id")

0       15_7474_3_152_02_211030-180248_bem_8cc_elicit_...
1       10_2345_0_101_01_211028-193045_bem_ddd_elicit_...
2       34_5026_3_342_02_211113-233703_bem_50c_elicit_...
3             149_2_01_210928-115147_bem_dcc_elicit_1.wav
4       36_11404_1_362_02_220227-135700_bem_d0b_elicit...
                              ...                        
2774    28_3507_3_282_02_211111-013203_bem_665_elicit_...
2775    29_3801_2_291_01_211114-135901_bem_433_elicit_...
2776    29_3801_1_292_02_211114-101121_bem_a9e_elicit_...
2777    29_3801_4_291_01_211114-135901_bem_433_elicit_...
2778    29_3801_3_292_02_211114-101121_bem_a9e_elicit_...
Name: audio_id, Length: 2779, dtype: object

In [225]:
df_test.tail()

,audio,sentence,translation,speaker_id
2774,audio/28_3507_3_282_02_211111-013203_bem_665_e...,Ububwangalo buntu aleteya kwena nangula ulifye...,Bonanza and other alike games can be exciting ...,282
2775,audio/29_3801_2_291_01_211114-135901_bem_433_e...,kena ifyo ifyacimana ukukula takalefimona na u...,For her she is not even paying attention to ho...,291
2776,audio/29_3801_1_292_02_211114-101121_bem_a9e_e...,umumana wine uyu uukulu saana nangula aka kamw...,This river is quite big even if this girl is n...,292
2777,audio/29_3801_4_291_01_211114-135901_bem_433_e...,ndemona kafwile kali nabamo abakulu abalekamon...,I am sure she is with someone older who is loo...,291
2778,audio/29_3801_3_292_02_211114-101121_bem_a9e_e...,"abaice nabo,pantu umukuku ngawalishile kukasen...",If a strong wind came she can easily be carrie...,292


# Check Null

In [226]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82371 entries, 0 to 82370
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   audio        82371 non-null  object
 1   sentence     82371 non-null  object
 2   translation  82371 non-null  object
 3   speaker_id   82371 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [227]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782 entries, 0 to 2781
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   audio        2782 non-null   object
 1   sentence     2782 non-null   object
 2   translation  2782 non-null   object
 3   speaker_id   2782 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 87.1+ KB


In [228]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2779 entries, 0 to 2778
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   audio        2779 non-null   object
 1   sentence     2779 non-null   object
 2   translation  2779 non-null   object
 3   speaker_id   2779 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 87.0+ KB


# Check Duplicates

In [229]:
df_train_duplicate = df_train[df_train["sentence"].duplicated(keep=False)]

In [230]:
df_train_duplicate[df_train_duplicate["sentence"]=="Umwaume aleniina ulupili"]

,audio,sentence,translation,speaker_id
20439,audio/8_5685_0_81_01_211102-100839_bem_ce8_eli...,Umwaume aleniina ulupili,A man is climbing a mountain.,81
81718,audio/8_1908_0_81_01_211031-092558_bem_abe_eli...,Umwaume aleniina ulupili,A man climbing a mountain.,81


#### Conclusion: all of the audio is different, even though the speaker_id is the same

# Check Overlap

In [231]:
df_val_train_overlap = df_val[df_val["sentence"].isin(df_train["sentence"].unique().tolist())]

In [232]:
df_val_train_overlap.head()

,audio,sentence,translation,speaker_id
120,audio/12_2751_0_121_01_211024-133017_bem_bd6_e...,bashikulu bantu baleendela pabwato,An old man is on a boat moving,121
201,audio/8_5533_1_82_02_211104-222444_bem_901_eli...,Cilepala kwati fibusa,They seem to be friends.,82
265,audio/12_2970_0_121_01_211101-025359_bem_bd6_e...,imbwa shibili shileangala,dogs are playing,121
555,audio/762_1_02_211011-173036_bem_81c_elicit_0.wav,Umukashana alemoneka ekele fye mwenso mwenso.,The lady looks scared,12
720,audio/5_1168_0_51_01_211102-090024_bem_e3a_eli...,Imbwa ili mumpanga.,a dog is in the Forest,51


In [233]:
df_val_train_overlap.shape

(25, 4)

In [234]:
df_val_test = df_val[df_val["sentence"].isin(df_test["sentence"].unique().tolist())]

In [235]:
df_val_test.head()

,audio,sentence,translation,speaker_id


In [236]:
df_train_val = df_train[df_train["sentence"].isin(df_val["sentence"].unique().tolist())]

In [237]:
df_train_val.shape

(51, 4)

In [238]:
df_train_test = df_train[df_train["sentence"].isin(df_test["sentence"].unique().tolist())]

In [239]:
df_train_test.shape

(45, 4)

In [240]:
df_test_train = df_test[df_test["sentence"].isin(df_train["sentence"].unique().tolist())]

In [241]:
df_test_train.shape

(16, 4)

In [242]:
df_test_train.head()

,audio,sentence,translation,speaker_id
167,audio/8_1925_0_81_01_211031-100405_bem_abe_eli...,Umwaume alecofa incinga,A young man is riding a bicycle.,81
378,audio/32_10393_1_322_02_220324-002458_bem_c25_...,Bushe uyu umwaume no mwanakashi fyakufwala nsh...,What clothes are this man and woman wearing?,322
448,audio/8_5696_0_81_01_211102-102600_bem_ce8_eli...,Umulumendo ali pamenshi,A young man is in water.,81
630,audio/1_20_1_12_02_211025-222323_bem_d90_elici...,Bushe finshi filecitika?,What exactly is happening?,12
739,audio/330_0_01_211015-235340_bem_9d4_elicit_0.wav,Umukashana ali mu muputule wa kwipikilamo.,A young girl is in the kitchen.,11


In [243]:
df_train_test[df_train_test["sentence"]=="Bushe finshi filecitika?"]

,audio,sentence,translation,speaker_id
34580,audio/35_11049_1_352_02_220117-183806_bem_f12_...,Bushe finshi filecitika?,What are they doing?,352
41667,audio/9_2006_2_91_01_211024-163655_bem_ebf_eli...,Bushe finshi filecitika?,What is happening?,91
45134,audio/1_31_1_12_02_211026-093457_bem_d90_elici...,Bushe finshi filecitika?,what exactly is happening?,12
52485,audio/9_5872_2_91_01_211030-101512_bem_901_eli...,Bushe finshi filecitika?,what are they doing?,91
69659,audio/35_10964_3_352_02_220117-170336_bem_f12_...,Bushe finshi filecitika?,What is happening?,352


In [244]:
df_test_val

,audio_id,sentence,translation,speaker_id


### Removing test data that is overlapped with train data

In [245]:
df_test_clean = df_test[~df_test["sentence"].isin(df_train["sentence"].unique().tolist())].reset_index(drop=True)

In [246]:
df_test_clean.head()

,audio,sentence,translation,speaker_id
0,audio/15_7474_3_152_02_211030-180248_bem_8cc_e...,nafwala na amakalashi ku menso,he is wearing glasses as well,152
1,audio/10_2345_0_101_01_211028-193045_bem_ddd_e...,"Imbwa iyafonka pamoona, ilebutuka palunkoto lw...",A dog with a wide nose is running on the lawns...,101
2,audio/34_5026_3_342_02_211113-233703_bem_50c_e...,Akamwanakashi nakemya ukuulu mumuulu ukulwisha...,She has her leg in the air attempting to break...,342
3,audio/149_2_01_210928-115147_bem_dcc_elicit_1.wav,Kunuma yabo kuli notu ma motoka tulya ba bonfy...,There are also small vehicles that they use in...,11
4,audio/36_11404_1_362_02_220227-135700_bem_d0b_...,abaume Bali pa mutenge yanganda umo afwele ish...,"Of the men on the roof of the house, ine is we...",362


In [247]:
df_test_clean.shape[0] == (df_test.shape[0] - 16)

True

In [248]:
df_val.shape

(2782, 4)

In [249]:
df_val.tail()

,audio,sentence,translation,speaker_id
2777,audio/19_1282_2_191_01_211108-211219_bem_665_e...,"Aba banensu basompa imyona nabo, uyu muntu imi...","He has a pointed nose, he looks like a lady",191
2778,audio/12537_4_01_211102-173234_bem_142_elicit_...,Nibamona mune ala uyu umupungu walilelema kabi...,"I have seen it, the rice looks and green, this...",11
2779,audio/12537_3_02_211102-173208_bem_dcc_elicit_...,Mukumona kwandi mupunga uyu balebyala kumona f...,"This should be rice, it's clear because it's h...",12
2780,audio/12537_2_01_211102-173234_bem_142_elicit_...,Cishinka sana balebomfye n'gombe pakulima kabi...,Yes they are using cows for ploughing and this...,11
2781,audio/12537_1_02_211102-173208_bem_dcc_elicit_...,Abantu aba balebomfye n'gombe pakulima elo in'...,These people are using very fat cows for ploug...,12


# Save to CSV 

In [250]:
df_train.to_csv("train_v1.csv", index=False)
df_val.to_csv("val_v1.csv", index=False)
df_test.to_csv("test_v1.csv", index=False)

In [251]:
df_test_clean.to_csv("test_v1_no_overlap.csv", index=False)

# Push it to HF

In [254]:
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test_clean)

In [255]:
train_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 82371
})

In [256]:
val_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 2782
})

In [257]:
test_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 2763
})

In [258]:
#convert to Audio
train_dataset = train_dataset.cast_column("audio", Audio())
val_dataset = val_dataset.cast_column("audio", Audio())
test_dataset = test_dataset.cast_column("audio", Audio())

In [259]:
test_dataset[0]["audio"]

{'path': 'audio/15_7474_3_152_02_211030-180248_bem_8cc_elicit_1.wav',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.15527344e-05, -6.10351562e-05, -6.10351562e-05]),
 'sampling_rate': 16000}

In [260]:
dataset_dict = DatasetDict({
    "train": train_dataset,
    "val": val_dataset,
    "test": test_dataset
})

In [262]:
from huggingface_hub import notebook_login

notebook_login()

In [263]:
from huggingface_hub import whoami

print(whoami()) 

{'type': 'user', 'id': '6573e3d7244aefdfc476f07a', 'name': 'cobrayyxx', 'fullname': 'M Hazim AF', 'email': 'muhammadhazim86@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/16ef7a7d5351781180384c7f5b4ef392.svg', 'orgs': [{'type': 'org', 'id': '643e51141d0e956d94baaade', 'name': 'kreasof-ai', 'fullname': 'Kreasof AI', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/63b6f2e752c02ae8acbaa4d8/r_B-tRS854Qo5EOURHxL4.png', 'roleInOrg': 'write', 'isEnterprise': False}, {'type': 'org', 'id': '67a769f44edabdd17b5c169b', 'name': 'SpeechT', 'fullname': 'SpeechT', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/618c42e4f35289d4bcab9cf1/m1k2sALFEwa9B2McXNyOi.png', 'roleInOrg': 'read', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'colab', 'role': 'writ

In [264]:
dataset_dict.push_to_hub("kreasof-ai/be-en-IWSLT2025")

Uploading the dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

Map:   0%|          | 0/2010 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/21 [00:00<?, ?ba/s]

Map:   0%|          | 0/2010 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/21 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/1382 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

Map:   0%|          | 0/1381 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kreasof-ai/be-en-IWSLT2025/commit/e182c1b4656d0415f26317f7f96cb04f8ce3986c', commit_message='Upload dataset', commit_description='', oid='e182c1b4656d0415f26317f7f96cb04f8ce3986c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kreasof-ai/be-en-IWSLT2025', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kreasof-ai/be-en-IWSLT2025'), pr_revision=None, pr_num=None)